# Biomaker CA: performing runs on a configuration

In this colab we show how to run models on a configuration and how to evaluate them.

We also show how to extract some agents at the end of a run and how to evaluate them in a fresh run.

This colab is consistent with the [original paper](https://arxiv.org/abs/2307.09320). It is not forward-compatible, so make sure to install the git tagged version 'biomaker-v1.0.0'.

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Imports

In [ ]:
#@title install selforg package
# install the package locally
!pip install --upgrade -e git+https://github.com/google-research/self-organising-systems.git@biomaker-v1.0.0#egg=self_organising_systems&subdirectory=biomakerca
# activate the locally installed package (otherwise a runtime restart is required)
import pkg_resources
import importlib
# Reload the resources because we uninstalled and reinstalled some packages.
importlib.reload(pkg_resources)
pkg_resources.get_distribution("self_organising_systems").activate()

In [ ]:
#@title imports & notebook utilities
from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.mutators import BasicMutator
from self_organising_systems.biomakerca.mutators import RandomlyAdaptiveMutator
from self_organising_systems.biomakerca.step_maker import step_env
from self_organising_systems.biomakerca.display_utils import zoom
from self_organising_systems.biomakerca.custom_ipython_display import display
from self_organising_systems.biomakerca.env_logic import ReproduceOp
from self_organising_systems.biomakerca.env_logic import env_perform_one_reproduce_op

import cv2
import numpy as np
import jax.random as jr
import jax.numpy as jp
from jax import vmap
from jax import jit
import jax
import time

import tqdm
import mediapy as media
from functools import partial


def pad_text(img, text):
  font = cv2.FONT_HERSHEY_SIMPLEX
  orgin = (5, 15)
  fontScale = 0.5
  color = (0, 0, 0)
  thickness = 1

  # ensure to preserve even size (assumes the input size was even.
  new_h = img.shape[0]//15
  new_h = new_h if new_h % 2 == 0  else new_h + 1
  img = np.concatenate([np.ones([new_h, img.shape[1], img.shape[2]]), img], 0)
  img = cv2.putText(img, text, orgin, font, fontScale, color, thickness, cv2.LINE_AA)
  return img

## Select the configuration, the agent logic and the mutator

Set soil_unbalance_limit to 0 to reproduce the original environment. Set it to 1/3 for having self-balancing environments (recommended).

In [ ]:
ec_id = "pestilence" #@param ['persistence', 'pestilence', 'collaboration', 'sideways']
env_width_type = "landscape" #@param ['wide', 'landscape', 'square', 'petri']
soil_unbalance_limit = 0 #@param [0, "1/3"] {type:"raw"}

env_and_config = evm.get_env_and_config(ec_id, width_type=env_width_type)
st_env, config = env_and_config
config.soil_unbalance_limit = soil_unbalance_limit

agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "basic" #@param ['basic', 'randomly_adaptive']
sd = 1e-2 if mutator_type == "basic" and agent_model == "basic" else 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

## Optionally, modify the config for custom configurations

In [ ]:
print("Current config:")
print('\n'.join("%s: %s" % item for item in vars(config).items()))

In [ ]:
## Examples for modifying the config
## Uncomment relevant lines or do like them.

## Regardless, to trigger the recomputation of step_env and similar,
## config needs to be a new object! So, first, we create a new copy.
import copy
config = copy.copy(config)

## Change simple isolated parameters (most of them)
# config.struct_integrity_cap = 100
# config.max_lifetime = 500
## Vectors can be modified either by writing new vectors:
# config.dissipation_per_step = jp.array([0.02, 0.02])
## Or by multiplying previous values. Note that they are immutable!
# config.dissipation_per_step = config.dissipation_per_step * 2

## agent_state_size is trickier, because it influences env_state_size.
## So you can either create a new config:
## Note that you would have to insert all values that you don't want to take
## default initializations.
# config = evm.EnvConfig(agent_state_size=4)
## Or you can just modify env_state_size as well.
## (env_state_size = agent_state_size + 4) for now.
# config.agent_state_size = 4
# config.env_state_size = config.agent_state_size + 4


## Perform a simulation

Consider modifying the code to vary the extent of the simulation and video configs.

In [ ]:

key = jr.PRNGKey(43)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

# if True, every 50 steps we check whether the agents go extinct. If they did,
# we replace a seed in the environment.
replace_if_extinct = False

# The number of frames of the video. This is NOT the number of steps.
# The total number of steps depend on the number of steps per frame, which can
# vary over time.
# In the article, we generally use 500 or 750 frames.
n_frames = 500

# on what FRAME to double speed.
when_to_double_speed = [100, 200, 300, 400, 500]
# on what FRAME to reset speed.
when_to_reset_speed = []
fps = 20
# this affects the size of the image. If this number is not even, the resulting
# video *may* not be supported by all renderers.
zoom_sz = 4

ku, key = jr.split(key)
programs = vmap(agent_logic.initialize)(jr.split(ku, N_MAX_PROGRAMS))
ku, key = jr.split(key)
programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)

env = st_env


def make_frame(env, step, speed):
  return pad_text(zoom(evm.grab_image_from_env(env, config),zoom_sz),
                  "Step {:<7} Speed: {}x".format(step, speed))

step = 0
# how many steps per frame we start with. This gets usually doubled many times
# during the simulation.
# In the article, we usually use 2 or 4 as the starting value, sometimes 1.
steps_per_frame = 2

frame = make_frame(env, step, steps_per_frame)

out_file = "video.mp4"
with media.VideoWriter(
    out_file, shape=frame.shape[:2], fps=fps, crf=18) as video:
  video.add_image(frame)
  for i in tqdm.trange(n_frames):
    if i in when_to_double_speed:
      steps_per_frame *= 2
    if i in when_to_reset_speed:
      steps_per_frame = 1
    for j in range(steps_per_frame):
      step += 1
      key, ku = jr.split(key)
      env, programs = step_env(
          ku, env, config, agent_logic, programs, do_reproduction=True,
            mutate_programs=True, mutator=mutator)
      if replace_if_extinct and step % 50 == 0:
        # check if there is no alive cell.
        any_alive = jit(lambda type_grid: evm.is_agent_fn(type_grid).sum() > 0)(env.type_grid)
        if not any_alive:
          # Then place a new seed.
          agent_init_nutrients = (config.dissipation_per_step * 4 +
                                  config.specialize_cost)
          ku, key = jr.split(key)
          rpos = jp.stack([0, jr.randint(ku, (),minval=0, maxval=env.type_grid.shape[1])],0)
          ku, key = jr.split(key)
          raid = jr.randint(ku, (),minval=0, maxval=N_MAX_PROGRAMS).astype(jp.uint32)
          repr_op = ReproduceOp(1., rpos, agent_init_nutrients*2, raid)
          ku, key = jr.split(key)
          env = jit(partial(env_perform_one_reproduce_op, config=config))(ku, env, repr_op)

          # show it, though
          frame = make_frame(env, step, steps_per_frame)
          for stop_i in range(10):
            video.add_image(frame)

    video.add_image(make_frame(env, step, steps_per_frame))

media.show_video(media.read_video(out_file))

## Evaluate the configuration

With this code, you can either evaluate a randomly initialized model, or models extracted from the previous run.

In the latter case, make sure that there are agents alive at the end of the simulation.

In [ ]:
def count_agents_f(env, etd):
  return etd.is_agent_fn(env.type_grid).sum()

@partial(jit, static_argnames=["config", "agent_logic", "mutator", "n_steps", "n_max_programs"])
def evaluate_biome(key, st_env, config, agent_logic, mutator, n_steps,
                   init_program=None, n_max_programs=128):
  def body_f(i, carry):
    key, env, programs, tot_agents_n = carry
    ku, key = jr.split(key)

    env, programs = step_env(
        ku, env, config, agent_logic, programs, do_reproduction=True,
          mutate_programs=True, mutator=mutator)

    tot_agents_n += count_agents_f(env, config.etd)
    return key, env, programs, tot_agents_n

  if init_program is None:
    ku, key = jr.split(key)
    programs = vmap(agent_logic.initialize)(jr.split(ku, n_max_programs))
    ku, key = jr.split(key)
    programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)
  else:
    programs = jp.repeat(init_program[None, :], n_max_programs, axis=0)

  key, env, programs, tot_agents_n = jax.lax.fori_loop(
      0, n_steps, body_f, (key, st_env, programs, 0))

  # check whether they got extinct:
  is_extinct = (count_agents_f(env, config.etd) == 0).astype(jp.int32)
  return tot_agents_n, is_extinct

In [ ]:
what_to_evaluate = "initialization" # @param ["initialization", "extracted"]

key = jr.PRNGKey(123)

n_steps = 1000
n_reps = 16

if what_to_evaluate == "initialization":
  init_programs = None
else:
  # Extract a living program from the final environment.
  aid_flat = env.agent_id_grid.flatten()
  is_agent_flat = evm.is_agent_fn(env.type_grid).flatten().astype(jp.float32)
  n_alive_per_id = jax.ops.segment_sum(is_agent_flat, aid_flat, num_segments=N_MAX_PROGRAMS)
  alive_programs = programs[n_alive_per_id>0]
  print("Extracted {} programs.".format(alive_programs.shape[0]))
  assert alive_programs.shape[0] >= n_reps, "Not enough alive programs found."

  init_programs = alive_programs[:n_reps]


t_st = time.time()
key, ku = jr.split(key)
b_tot_agents_n, b_is_extinct = jit(vmap(partial(
    evaluate_biome, st_env=st_env, config=config, agent_logic=agent_logic,
    mutator=mutator, n_steps=n_steps)))(jr.split(ku, n_reps), init_program=init_programs)
print("Took", time.time()-t_st, "seconds")
print("Total number of agents", b_tot_agents_n, b_tot_agents_n.mean(), b_tot_agents_n.std())
print("Extinction events", b_is_extinct, b_is_extinct.mean(), b_is_extinct.std())